## Test for the round trip

This notebook tests the round trip. The initial primary key output by the create-round-trip-data notebook needs to be provided.

In [31]:
import boto3
from botocore.config import Config

In [32]:
# Constants
table_name = 'round-trip-table'
initial_primary_key = "038aca33-a8c0-4b5d-8543-cc50b8f4895c"

In [33]:
# Connect to DynamoDB using the boto3 client
botoConfig = Config(
    region_name = 'eu-central-1',
    signature_version = 'v4',
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)
dynamodb = boto3.client('dynamodb', config=botoConfig)

In [34]:
# Check if the table exists
response = dynamodb.list_tables()
if table_name not in response['TableNames']:
    Exception("Table does not exist")
else:
    print("Table exists")

Table exists


In [35]:
# We can now test the round trip by querying the table
current_primary_key = initial_primary_key
counter = 0
while True:
    counter += 1
    response = dynamodb.get_item(
        TableName=table_name,
        Key={
            'primary_key': {'S': current_primary_key}
        }
    )
    current_primary_key = response['Item']['next_primary_key']['S']
    if current_primary_key == initial_primary_key:
        break

print("The round trip took " + str(counter) + " queries")

The round trip took 200 queries
